In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_array = np.random.random((1000,10))
print(train_array)

[[ 0.55258167  0.8189179   0.08171256 ...,  0.30631806  0.60036536
   0.56155825]
 [ 0.86859212  0.64916329  0.35781675 ...,  0.0380998   0.80314577
   0.71295245]
 [ 0.9279222   0.74499459  0.63155931 ...,  0.2251169   0.40717691
   0.22885483]
 ..., 
 [ 0.02480216  0.24798578  0.51897618 ...,  0.8340184   0.55352014
   0.95838871]
 [ 0.28831181  0.26763278  0.56415768 ...,  0.91126467  0.08775347
   0.62718961]
 [ 0.80188667  0.28764395  0.00680379 ...,  0.66796126  0.47150098
   0.24616637]]


In [3]:
train_rows = np.shape(train_array)[0]
train_cols = np.shape(train_array)[1]
train_labels = np.zeros((train_rows,train_cols))
for i in range(train_rows):
    for j in range(train_cols):
        if j == 0:
            train_labels[i,j] = max(train_array[i,:])
        else:
            train_labels[i,j] = train_labels[i,(j-1)]-0.01
print(train_labels)

[[ 0.8189179   0.8089179   0.7989179  ...,  0.7489179   0.7389179
   0.7289179 ]
 [ 0.86859212  0.85859212  0.84859212 ...,  0.79859212  0.78859212
   0.77859212]
 [ 0.9279222   0.9179222   0.9079222  ...,  0.8579222   0.8479222
   0.8379222 ]
 ..., 
 [ 0.95838871  0.94838871  0.93838871 ...,  0.88838871  0.87838871
   0.86838871]
 [ 0.9261799   0.9161799   0.9061799  ...,  0.8561799   0.8461799
   0.8361799 ]
 [ 0.88216755  0.87216755  0.86216755 ...,  0.81216755  0.80216755
   0.79216755]]


In [4]:
def mlp(x, hidden_sizes, activation_fn=tf.nn.relu):
    if not isinstance(hidden_sizes, (list, tuple)):
        raise ValueError("hidden_sizes must be a list or a tuple")
    W = {}
    b = {}
    h = {}
    for i in range(len(hidden_sizes)):
        if(W == {}):
            W["W"+str(i)] = tf.get_variable("W"+str(i),[1000,hidden_sizes[i]],tf.float32,tf.random_normal_initializer(stddev = 0.01))
            b["b"+str(i)] = tf.get_variable("b"+str(i),[hidden_sizes[i]],tf.float32,tf.constant_initializer(0.0))
            h["h"+str(i)] = activation_fn(tf.matmul(x,W["W"+str(i)])+b["b"+str(i)])
        elif(i == len(hidden_sizes)-1):
            W["W"+str(i)] = tf.get_variable("W"+str(i),[hidden_sizes[i-1],hidden_sizes[i]],tf.float32,tf.random_normal_initializer(stddev = 0.01))
            b["b"+str(i)] = tf.get_variable("b"+str(i),[hidden_sizes[i]],tf.float32,tf.constant_initializer(0.0))
            return (tf.matmul(h["h"+str(i-1)],W["W"+str(i)])+b["b"+str(i)])
        else:
            W["W"+str(i)] = tf.get_variable("W"+str(i),[hidden_sizes[i-1],hidden_sizes[i]],tf.float32,tf.random_normal_initializer(stddev = 0.01))
            b["b"+str(i)] = tf.get_variable("b"+str(i),[hidden_sizes[i]],tf.float32,tf.constant_initializer(0.0))
            h["h"+str(i)] = activation_fn(tf.matmul(h["h"+str(i-1)],W["W"+str(i)])+b["b"+str(i)])

In [8]:
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32,[None,1000])
    y = tf.placeholder(tf.float32,[None,10])
    prediction = mlp(x,[1000,512,10])
    loss = tf.reduce_sum(tf.square(prediction-y))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(1000):
            loss_avg = 0
            for data in range(1000):
                x_train = np.reshape(train_array[data,:],(1,10))
                y_train = train_labels[data,:]
                for i in range(99):
                    y_train = np.hstack((y_train,train_labels[data,:]))
                y_train = np.reshape(y_train,(1,1000))
                
                #print(x_train,y_train)
                _,loss_here = sess.run([optimizer,loss],feed_dict = {x:y_train,y:x_train})
                #print("loss:",loss)
                loss_avg += loss_here
            print(loss_avg/10)
        x_test = np.random.random((1,10))
        test_label = np.zeros((np.shape(x_test)[0],np.shape(x_test)[1]))
        for i in range(np.shape(x_test)[0]):
            for j in range(np.shape(x_test)[1]):
                if j == 0:
                    test_label[i,j] = max(x_test[i,:])
                else:
                    test_label[i,j] = test_label[i,(j-1)]-0.01
        print(x_test)
        print(sess.run([prediction],feed_dict = {x:test_label,y:y_train}))
        #sess.run([optimizer],)

89.3151436634
84.7270083636
84.0200589031
83.7259195596
83.5302255228
83.3761953652
83.2317996055
83.0709823787
82.9251711085
82.8035064682
82.6875219047
82.6123029545
82.5722400934
82.5486056566
82.514804703
82.4833334148
82.4687740833
82.453437534
82.4347375378
82.4006348252
82.367153953
82.3535273612
82.3458350435
82.3218339249
82.2896307632
82.2597631514
82.2544191331
82.2506667122
82.2475058302
82.2445831284
82.2417691082
82.2390153438
82.2363052398
82.2336299792
82.2309869856
82.228373611
82.2257881165
82.2232265934
82.2206867471
82.2181638777
82.2156515062
82.2131455392
82.2106375962
82.2081162736
82.2055723071
82.2029901028
82.2003538176
82.1976401374
82.1948243603
82.1918723434
82.1887419716
82.1853774831
82.1817110747
82.1776458025
82.1730612889
82.1677919149
82.1616254896
82.154304035
82.1468196064
82.1388477743
82.1224761426
82.0933434263
82.0868695512
82.080368343
82.074048768
82.0678825617
82.0618611991
82.0559793383
82.0502356336
82.0446268454
82.039154309
82.0338176146


KeyboardInterrupt: 